In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-20 12:55:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

100%[======================================>] 1,115,394   --.-K/s   in 0.04s   

2024-02-20 12:55:08 (28.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


## unique characters and mapping the characters to integer

In [32]:
chars = sorted(set(text))
vocab_size = len(chars)
print('characters', chars)

characters ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[s] for s in x]
decode = lambda x: "".join([itos[s] for s in x])

print(encode("Hi there! how are you doing")) 
print(decode(encode("Hi there! how are you doing")))

[20, 47, 1, 58, 46, 43, 56, 43, 2, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59, 1, 42, 53, 47, 52, 45]
Hi there! how are you doing


In [7]:
# moving data to torch tensor

In [8]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### converting the text into train test splits

In [9]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_data, val_data = train_test_split(data, test_size=0.1, shuffle=False)

In [12]:
block_size = 8 
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input : {context}, output : {target}")

input : tensor([18]), output : 47
input : tensor([18, 47]), output : 56
input : tensor([18, 47, 56]), output : 57
input : tensor([18, 47, 56, 57]), output : 58
input : tensor([18, 47, 56, 57, 58]), output : 1
input : tensor([18, 47, 56, 57, 58,  1]), output : 15
input : tensor([18, 47, 56, 57, 58,  1, 15]), output : 47
input : tensor([18, 47, 56, 57, 58,  1, 15, 47]), output : 58


In [14]:
torch.manual_seed(123)

In [15]:
batch_size = 4
block_size = 8

In [19]:
len(data)

1115394

In [18]:
torch.randint(len(data) - block_size, (batch_size,)) # creating a batch of data

tensor([ 587602,  401082, 1029794,  223223])

In [23]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [24]:
xb, yb = get_batch('train')

In [25]:
print('inputs')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs
torch.Size([4, 8])
tensor([[ 1, 44, 56, 53, 51,  1, 51, 63],
        [57,  6,  1, 58, 46, 43, 63,  1],
        [46, 43,  1, 42, 47, 52,  1, 53],
        [50, 43, 58,  1, 58, 46, 43, 51]])
targets:
torch.Size([4, 8])
tensor([[44, 56, 53, 51,  1, 51, 63,  1],
        [ 6,  1, 58, 46, 43, 63,  1, 56],
        [43,  1, 42, 47, 52,  1, 53, 44],
        [43, 58,  1, 58, 46, 43, 51,  1]])


In [26]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [1] the target: 44
when input is [1, 44] the target: 56
when input is [1, 44, 56] the target: 53
when input is [1, 44, 56, 53] the target: 51
when input is [1, 44, 56, 53, 51] the target: 1
when input is [1, 44, 56, 53, 51, 1] the target: 51
when input is [1, 44, 56, 53, 51, 1, 51] the target: 63
when input is [1, 44, 56, 53, 51, 1, 51, 63] the target: 1
when input is [57] the target: 6
when input is [57, 6] the target: 1
when input is [57, 6, 1] the target: 58
when input is [57, 6, 1, 58] the target: 46
when input is [57, 6, 1, 58, 46] the target: 43
when input is [57, 6, 1, 58, 46, 43] the target: 63
when input is [57, 6, 1, 58, 46, 43, 63] the target: 1
when input is [57, 6, 1, 58, 46, 43, 63, 1] the target: 56
when input is [46] the target: 43
when input is [46, 43] the target: 1
when input is [46, 43, 1] the target: 42
when input is [46, 43, 1, 42] the target: 47
when input is [46, 43, 1, 42, 47] the target: 52
when input is [46, 43, 1, 42, 47, 52] the target: 1
when

In [27]:
print(xb)

tensor([[ 1, 44, 56, 53, 51,  1, 51, 63],
        [57,  6,  1, 58, 46, 43, 63,  1],
        [46, 43,  1, 42, 47, 52,  1, 53],
        [50, 43, 58,  1, 58, 46, 43, 51]])


## Creating a bigram language model 

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(123)

In [29]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) #B, T, C (batch size, block size and the embedding vector)

        if targets is None:
            loss=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) # B, C 
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
        return idx

In [33]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

In [34]:
m

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 65)
)

In [35]:
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.2654, grad_fn=<NllLossBackward0>)


In [38]:
torch.zeros((1, 1), dtype=torch.long)

tensor([[0]])

In [43]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


YxIadj!wm q-grdc-a;Zl.WAcIqcyemMq3odYPLX!nNH!bU&pDNquX
H:WTBC'EyFr-E!q SGJ:Ojn!jYr!NyLTqnSbUS-;-b;RJ


In [44]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for steps in range(100):
    # sample a batch of data
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.438876628875732


In [46]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


mSTNKGD3Psr&HBWbTLekoZtt.hyLz!w&uvHPtDvXLDmKh.RQi'T:i?inL
J-XByDPcln!EoMj:IFEttgN?-?J bngMJkgX!b;3mMUkrIJGsb;BwTzCzxwbkraN?jXI-t;epd'KUXueEJ'jeHLLE!gSgPttzv&zQlsKm3rFetdoPQwOS-EriO.?.Of!'.O aXKf?xnAwiIf.?stZ.bGDGznFQ3BBtXIANdvrZnn&fLzIUsm qtCyahyC,ggn .nF kXk'khMaZQu
khhKdUHHB'pcvQh.ccK?LX
K?:VT?aY llpPtLPcmD,TuAhiRZZP'C-mL:WFlb'OVg:WRzuvPL
KWaGDsb;SKA-aU3BoPzLpJSgsu-:WjM&VcL&OPLEmMohAzSgs aswt
rrD E
ZzWj!mK,SBtAZCRiLQN3dce.L
rFZNdVmtgceX$wgQRQ!c, YmTdfCXm aGs!gUL&VSQiftgW'BxUJKlhDSHr
GYkgp: YHF


## sel 